## Capstone Project -  Battle of the Neighborhoods_finding the best location for a restaurant

### Applied Data Science Project by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>

According to statistics, **40% of new business ventures fail within their first year**. One of the reasons that have been linked to these failures is **inadequate market research** by the new venture, which relates to the business understanding their potential clients that they will be serving, the potential competition and gaps in the market where they are trying to enter.
Understanding the market can help evaluate the decision of whether to go ahead with a business or not as well as the best way to take advantage of market gaps.

This notebook attempts to perform an important portion of market research, which is to **identify the best location for a burger restaurant** in the Northern suburbs of **Johannesburg, South Africa**. 
From research and discussions with the client, it is apparent that two important factors have to be met in order for a location to be considered suitable for a burger restaurant:

**a)	Youth driven Demographics:**

Areas where there are teenagers and young adults would be suitable     locations. This could be identified by high schools, colleges and clubs. 

**b)	Middle to high income households**

Middle to high income households neighborhoods are the most desirable as they are where  the residents will afford to have or buy restaurant meals on a regular basis. 

**c)	Low number of competitors in the area**

There must be a low number of burger restaurants in the area. A high number of restaurants in general may give an indication that the restaurant scene in the area is vibrant, but low number of burger restaurants is what is desired (i.e., areas where the least common restaurants are burger restaurants). 

The problem statement is then to determine the areas with the best demographics (high schools, colleges and clubs, as well as middle to high income households) that have the lowest number of burger restaurants.


## Data <a name="data"></a>

In order to determine the best location for the burger restaurant, **clustering** algorithm will be used to **group** suitable neighborhoods where burger restaurants are the least common. 
The features of the data will be **economic status of the neighborhood (middle to upper income), the number of schools in the neighborhood as well as the type of restaurants in the area**, with a particular interest in the number of burger restaurants. 

The following data sets will therefore be required:

1)	Neighborhoods in Johannesburg, South Africa <br> 
2)	Economic data of the neighborhoods (and determination of most of the residence being above the median for low income -R12800 per month)<br> 
3)	High schools, colleges and clubs in the area<br>
4)	Restaurants in a neighborhood, ranked from most to least common<br>

The first dataset listed above can be obtained from the city of Johannesburg website www.joburg.gov.za with particular interest in the suburbs of region A, which is the northern region. The demographic data (high schools, colleges and clubs.), and the restaurants can be obtained from **Foursquare**. The economic data of the neighborhoods and will be obtained from www.statssa.gov, which is a government run national statistics website.


### Neighborhoods/ suburbs in Johannesburg

We will first scrape the government website www.joburg.gov.za to get suburbs we are interested in. The website groups the suburbs/neighborhoods into regions. Each region has a table, which contains the suburbs of interest.  <br>
We are interested in the suburbs in Region A.

let's first import and install te required libraries for scraping

In [2]:
!pip install bs4
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

We then request the webpage for region A, convert it to a BS format and extract the table of interest, which has the neighborhoods/suburbs we are looking for

In [31]:
link="https://www.joburg.org.za/about_/regions/Pages/Region%20A%20-%20Diepsloot%2c%20Midrand/suburbs.aspx"
joburg_burbs = requests.get(link).text

joburg_bs=BeautifulSoup(joburg_burbs,"html5lib")

joburg_table_bs=joburg_bs.find("table").td

joburg_table_bs

<td class="ms-rteTableEvenCol-5">​Airdlin, <br/><br/>Barbeque, Barbeque Downs, Barbeque Downs Extensions, Beverley, Beverley Ext.1 &amp; 2, Beverley Extensions, Bloubosrand Extension, Blue Hills, Blue Hills Extension, Bothasfontein 408-Jr Ext.408 , Brendavere, Broadacres, Broadacres Ext.1 &amp; 2, Broadacres Extensions, Broadacres Park Ext.1 &amp; 10, Buccleuch, <br/><br/>Carlswald, Carlswald Estate, Chartwell, Country View, Country View Extensions, Craigavon, Craigavon Ext.1, Crowthorne , <br/><br/>Dainfern, Dainfern Extensions, Dainfern Ridge, Diepsloot 388-Jr Ext.388, Diepsloot, Diepsloot Wes, Diepsloot Wes Extensions, <br/><br/>Ebony Park, Ebony Park Extensions, Erand, Erand Ext.1 &amp; 2, Erand Gardens Extensions, <br/><br/>Farmall, Farmall Ext.1, Fourways Extensions, <br/><br/>Glen Austin, Glen Austin Ext.1, Glen Nerine, Glenferness, Glenferness Ext.1 &amp; 2, Grand Central Ext.21, <br/><br/>Halfway Gardens Extensions, Halfway House, Halfway House Estate, Halfway House Ext.13 Hal

We then create a dataframe and fill it with the suburbs from the beautifulsoup file/table

In [32]:
suburbs_data = pd.DataFrame()

for suburbs_group in joburg_table_bs.contents:
    suburbs =str(suburbs_group.string)
    suburbs_array = suburbs.split(',')
    
    for suburbs in suburbs_array:
        ser = pd.Series(str(suburbs.strip()))
        suburbs_data=suburbs_data.append(ser, ignore_index=True)
        
suburbs_data.columns=["Suburbs"]
        #burbs_data=burbs_data.append(element)

suburbs_data

,Suburbs
0,​Airdlin
1,
2,None
3,None
4,Barbeque
...,...
167,Willaway
168,Willaway Extensions
169,Witkoppen
170,Witkoppen Extensions.​


We need to clean up the dataframe. This will include removal of empty rows, as well as removal of items which form different sections of the same neighboughhood(e.g. Witkoppen and Witkoppen Extensions)

In [33]:
#remove empty rows
suburbs_data.replace("",np.nan,inplace=True)
suburbs_data.replace("None",np.nan,inplace=True)
suburbs_data.dropna(axis=0,inplace=True)
suburbs_data.reset_index(drop=True,inplace=True)

In [34]:
suburbs_data

,Suburbs
0,​Airdlin
1,Barbeque
2,Barbeque Downs
3,Barbeque Downs Extensions
4,Beverley
...,...
109,Waterval Ext.2 & 4
110,Willaway
111,Willaway Extensions
112,Witkoppen


In [35]:
#prepare data frame to remove duplicate items(different parts of the same suburb)
#first remove extensions and joint regions

import string

suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.replace("Extensions",""))
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.replace("Extension",""))
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.replace("Ext.",""))
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.replace("&",""))
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.replace("-Jr",""))

suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:"".join(x))

suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.rstrip(string.digits))

suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.strip())

In [36]:
#remove extension numbers
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.rstrip(string.digits))
suburbs_data["Suburbs"] = suburbs_data["Suburbs"].apply(lambda x:x.strip())

In [37]:
suburbs_data

,Suburbs
0,​Airdlin
1,Barbeque
2,Barbeque Downs
3,Barbeque Downs
4,Beverley
...,...
109,Waterval
110,Willaway
111,Willaway
112,Witkoppen


In [38]:
#remove duplicate items
suburbs_data.drop_duplicates(subset=None, inplace=True)
suburbs_data

,Suburbs
0,​Airdlin
1,Barbeque
2,Barbeque Downs
4,Beverley
7,Bloubosrand
...,...
106,Vorna Valley
108,Waterval
110,Willaway
112,Witkoppen


In [40]:
#With the dataframe prepared, now we can remove duplicates suburbs
suburb=""

for i in range(len(suburbs_data)):
    if i>0:
        suburb=str(suburbs_data.iloc[i])
        if suburb.split()[1]==str(suburbs_data.iloc[i-1]).split()[1]:
            suburbs_data.iloc[i] = np.nan   
            
suburbs_data.dropna(axis=0,inplace=True)
suburbs_data.reset_index(drop=True,inplace=True)
pd.set_option('display.max_rows', None)
suburbs_data

,Suburbs
0,​Airdlin
1,Barbeque
2,Beverley
3,Bloubosrand
4,Blue Hills
5,Bothasfontein
6,Brendavere
7,Broadacres
8,Buccleuch
9,Carlswald


After evaluating the dataframe, it looks like we have to fix a few lines, as some names were not stored correctly

In [41]:
suburbs_data.iloc[1]="Barbeque Downs"

suburbs_data.iloc[17]="Erand Gardens"

suburbs_data.iloc[43]="Needwood"
suburbs_data.iloc[35]="Kyalami"

suburbs_data.drop([36],axis=0,inplace=True)
suburbs_data

,Suburbs
0,​Airdlin
1,Barbeque Downs
2,Beverley
3,Bloubosrand
4,Blue Hills
5,Bothasfontein
6,Brendavere
7,Broadacres
8,Buccleuch
9,Carlswald


In [42]:
#write the dataframe to CSV
suburbs_data.to_csv("Joburg Northern Suburbs.csv", index=False)

### Economic data of the neighbohoods

We will now get economic data of the neighborhoods from the www.statssa.gov . Luckily, statssa prvides a link where the required data can for each suburb can be obtained, however they don describe the suburb by name, rather by ward number. We therefore need to relate our list of suburbs to ward numbers, and then get eonomic data for each suburb with allocated ward number

The file for wards and suburbs belonging to each ward was obtained as a PDF and converted to an csv file, provided with this notebook as Ward regions and numbers.csv

Only wards for region A are considered

In [44]:
#read in file containing wards and regions in each ward
ward_numbers_df=pd.read_csv("Ward regions and numbers.csv")
ward_numbers_df

,Ward number,Region,Suburbs
0,77,A,"Ivory Park Ext 9, 12 & 13 Thulani Section, Chr..."
1,78,A,"Ivory Park Ext 5, 6 & 8 Limpopo Section, Ramap..."
2,79,A,"Ivory Park Mafelandawonye 1 & 3, Baghdad, Lusa..."
3,80,A,"Rabie Ridge Ext 1, 4 & 5 New Hani Section, Chr..."
4,92,A,"Randjespark, Halfway House Estates , Halfway H..."
5,93,A,"Lonehill, Sunninghill, Paulshof, Magaliessig, ..."
6,94,A,"Witpoort 406 -JR, Kyalami AH, Kyalami Park, Gl..."
7,95,A,"Diepsloot West Reception Area Ext 1, Diepsloot..."
8,96,A,"Dainfern, Chartwell AH, North Champagne Estate..."
9,110,A,"Halfway House, Halfway House Estates, Presiden..."


In [45]:
#Allocate ward numbers to the suburbs in our file
suburbs_data["Ward number"]=""

for i in range(len(suburbs_data)):
    
    for j in range(len(ward_numbers_df)):
        suburb_list = ward_numbers_df.iloc[j,2]
        ward = ward_numbers_df.iloc[j,0]
    
        if suburbs_data.iloc[i,0] in suburb_list:
            suburbs_data.iloc[i,1] = ward

suburbs_data

,Suburbs,Ward number
0,​Airdlin,
1,Barbeque Downs,132
2,Beverley,94
3,Bloubosrand,
4,Blue Hills,112
5,Bothasfontein,
6,Brendavere,
7,Broadacres,96
8,Buccleuch,
9,Carlswald,132


In [46]:
suburbs_data.to_csv("Joburg Northern Suburbs.csv", index=False)

We read in file downloaded for suburb economic data, which breaks down the data by salary bracket, and for each salary bracket, the percentage of the population earning in the bracket is shown

In [47]:
stats_data = pd.read_csv("City of Johannesburg wards and incomes.csv")
stats_data

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_
0,1.0,28.802810,20.378240,7.697991,12.388655,11.435419,9.490369,5.346751,2.850184,2.408092,2.914555,1.735870,1.138840
1,2.0,42.437671,33.419763,10.743195,18.755504,15.751556,13.482632,9.050603,6.418465,3.914135,3.363843,1.156723,1.098295
2,4.0,29.413110,17.294432,7.486233,11.988171,12.012390,10.281529,6.179856,3.106334,2.553138,1.200738,1.166471,1.147472
3,3.0,26.105977,18.317708,7.686507,10.803043,11.146904,9.908012,7.326628,3.507385,2.936745,1.141167,1.659734,1.179470
4,5.0,16.632767,11.980037,5.195066,7.492183,7.500051,6.309033,4.590058,3.243199,2.035663,1.438465,1.288280,1.179470
5,6.0,9.802089,6.992818,3.498928,4.725400,5.116801,3.728643,2.450324,1.652313,1.826730,1.345695,1.157247,1.159177
6,7.0,14.919658,6.687932,3.805555,6.175742,6.217976,8.100347,12.436793,9.138113,5.715037,3.327129,4.021539,3.062079
7,8.0,9.149896,6.284952,3.982839,5.941747,4.734274,4.641722,6.149080,6.691919,3.168895,2.891866,4.889187,2.854255
8,9.0,27.763203,6.694544,5.002187,11.649067,8.673838,13.392642,15.181679,12.684063,8.210061,5.585036,2.071681,2.519514
9,10.0,9.023372,3.702254,2.651957,4.737060,3.834159,5.849896,9.043133,10.145056,6.935871,5.222880,2.613904,4.585894


We merge the two dataframes to see the economic data for each of our suburbs

In [49]:
suburbs_with_incomes = pd.merge(stats_data,suburbs_data,left_on="Ward",right_on="Ward number")

suburbs_with_incomes.drop("Ward number",axis=1,inplace=True)
suburbs_with_incomes

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs
0,113.0,17.420040,9.542690,5.275922,9.593424,13.221822,9.763110,5.944800,3.280168,2.410970,1.456859,1.440009,2.018726,Diepsloot
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens
4,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Midridge Park
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset
7,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Ebony Park
8,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Kaalfontein
9,110.0,23.323577,11.936488,6.618919,9.181493,11.607724,9.586493,7.693076,5.759109,3.545893,2.278187,1.754495,1.518320,Glen Austin


In [51]:
# columns have to be re-allocated because for some reason, python couldnt recognise column names
suburbs_with_incomes.columns

Index(['Ward', 'No income', 'Bracket A_R 1 to R 400_',
       'Bracket B_R 401 to R 800_', 'Bracket C_R 801 to R 1 600_',
       'Bracket D_R 1 601 to R 3 200_', 'Bracket E_R 3 201 to R 6 400_',
       'Bracket F_R 6 401 to R 12 800_', 'Bracket G_R 12 801 to R 25 600_',
       'Bracket H_R 25 601 to R 51 200_', 'Bracket I_R 51 201 to R 102 400_',
       'Bracket J_R 102 401 to R 204 800_', 'Bracket K_R 204 801 or more_',
       'Suburbs'],
      dtype='object')

In [52]:
suburbs_with_incomes.columns=['Ward', 'No income', 'Bracket A_R 1 to R 400_',
       'Bracket B_R 401 to R 800_', 'Bracket C_R 801 to R 1 600_',
       'Bracket D_R 1 601 to R 3 200_', 'Bracket E_R 3 201 to R 6 400_',
       'Bracket F_R 6 401 to R 12 800_', 'Bracket G_R 12 801 to R 25 600_',
       'Bracket H_R 25 601 to R 51 200_', 'Bracket I_R 51 201 to R 102 400_',
       'Bracket J_R 102 401 to R 204 800_', 'Bracket K_R 204 801 or more_',
       'Suburbs']

In [54]:
#add all percentages ffrom columns R12800 and higher to see what percentage for each suburb ears more than R12800 
suburbs_with_incomes["Percentage of Salaries Greater than R12 800"] = suburbs_with_incomes[['Bracket G_R 12 801 to R 25 600_',
       'Bracket H_R 25 601 to R 51 200_', 'Bracket I_R 51 201 to R 102 400_',
       'Bracket J_R 102 401 to R 204 800_', 'Bracket K_R 204 801 or more_']].sum(axis=1)

suburbs_with_incomes

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800
0,113.0,17.420040,9.542690,5.275922,9.593424,13.221822,9.763110,5.944800,3.280168,2.410970,1.456859,1.440009,2.018726,Diepsloot,10.606731
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662
4,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Midridge Park,46.295662
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662
7,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Ebony Park,20.512656
8,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Kaalfontein,20.512656
9,110.0,23.323577,11.936488,6.618919,9.181493,11.607724,9.586493,7.693076,5.759109,3.545893,2.278187,1.754495,1.518320,Glen Austin,14.856002


We label those surbubs who have a 35% or more of the population earning more than R12800 as qualifying for consideration

In [56]:
suburbs_with_incomes["Suburb Status"] = suburbs_with_incomes["Percentage of Salaries Greater than R12 800"].apply(lambda x:  "Qualifies" if x>25 else "Does not qualify")
suburbs_with_incomes

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800,Suburb Status
0,113.0,17.420040,9.542690,5.275922,9.593424,13.221822,9.763110,5.944800,3.280168,2.410970,1.456859,1.440009,2.018726,Diepsloot,10.606731,Does not qualify
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662,Qualifies
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662,Qualifies
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662,Qualifies
4,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Midridge Park,46.295662,Qualifies
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662,Qualifies
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662,Qualifies
7,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Ebony Park,20.512656,Does not qualify
8,111.0,23.350987,10.921942,6.448615,9.428313,12.148321,13.337023,13.216892,10.395609,3.502225,2.288124,1.995837,2.330862,Kaalfontein,20.512656,Does not qualify
9,110.0,23.323577,11.936488,6.618919,9.181493,11.607724,9.586493,7.693076,5.759109,3.545893,2.278187,1.754495,1.518320,Glen Austin,14.856002,Does not qualify


In [58]:
suburbs_with_incomes[suburbs_with_incomes["Suburb Status"] =='Qualifies'].shape

(27, 16)

There are 27 suburbs that qualify for consideration in terms of economical conditions

In [59]:
qualifying_suburbs = suburbs_with_incomes[suburbs_with_incomes['Suburb Status'] == "Qualifies"]
qualifying_suburbs

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800,Suburb Status
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662,Qualifies
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662,Qualifies
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662,Qualifies
4,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Midridge Park,46.295662,Qualifies
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662,Qualifies
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662,Qualifies
11,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Broadacres,35.716891,Qualifies
12,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Chartwell,35.716891,Qualifies
13,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Dainfern,35.716891,Qualifies
14,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Farmall,35.716891,Qualifies


We now obtain the coordinates of the qualifying suburbs so that we can use forusquare to get venues in the vicinity of the surburbs. The coordinates are obtained using geopy. We define a function to call to obtain coordinates for each suburb

First, we install and import the required libraries

In [60]:
!pip install folium
import folium 

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Define function to obtain coordinates for each location in the dataframe

In [61]:
def get_lat_lng(x):
    geolocator = Nominatim(user_agent="Kagiso M")
    
    location = geolocator.geocode('{}, Johannesburg'.format(x))
    try:
        latitude = location.latitude
        longitude = location.longitude
        return [latitude, longitude]
    
    except:
        latitude = "Not available"
        longitude= "Not available"
        return np.nan

In [62]:
qualifying_suburbs['Latitude and Longitude'] = qualifying_suburbs['Suburbs'].apply(get_lat_lng)
qualifying_suburbs

C:\Users\kagiso\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800,Suburb Status,Latitude and Longitude
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662,Qualifies,"[-25.9382165, 28.1043655]"
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662,Qualifies,"[-25.945718, 28.120936100539183]"
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662,Qualifies,"[-25.9738276, 28.124027]"
4,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Midridge Park,46.295662,Qualifies,NaN
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662,Qualifies,"[-26.1178936, 28.1543285]"
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662,Qualifies,"[-25.96078, 28.1085453]"
11,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Broadacres,35.716891,Qualifies,"[-26.0001873, 27.9823352]"
12,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Chartwell,35.716891,Qualifies,"[-26.1675131, 28.052286206246144]"
13,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Dainfern,35.716891,Qualifies,"[-25.993149199999998, 28.004047232216983]"
14,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Farmall,35.716891,Qualifies,NaN


In [64]:
#drop suburbs that dont have coordinates
qualifying_suburbs.dropna(axis=0, inplace=True)


C:\Users\kagiso\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [66]:
qualifying_suburbs

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800,Suburb Status,Latitude and Longitude
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662,Qualifies,"[-25.9382165, 28.1043655]"
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662,Qualifies,"[-25.945718, 28.120936100539183]"
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662,Qualifies,"[-25.9738276, 28.124027]"
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662,Qualifies,"[-26.1178936, 28.1543285]"
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662,Qualifies,"[-25.96078, 28.1085453]"
11,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Broadacres,35.716891,Qualifies,"[-26.0001873, 27.9823352]"
12,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Chartwell,35.716891,Qualifies,"[-26.1675131, 28.052286206246144]"
13,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Dainfern,35.716891,Qualifies,"[-25.993149199999998, 28.004047232216983]"
15,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Kengies,35.716891,Qualifies,"[-25.9984801, 27.9975803]"
17,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Riverbend,35.716891,Qualifies,"[-26.0298442, 27.9620846]"


In [71]:
# separate the longitude and latitude into two separate columns

qualifying_suburbs["latitude"] = qualifying_suburbs["Latitude and Longitude"].apply(lambda x:x[0])
qualifying_suburbs["longitude"] = qualifying_suburbs["Latitude and Longitude"].apply(lambda x:x[1])

qualifying_suburbs.drop(labels="Latitude and Longitude",axis=1,inplace=True)

qualifying_suburbs

C:\Users\kagiso\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kagiso\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\kagiso\anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,Bracket I_R 51 201 to R 102 400_,Bracket J_R 102 401 to R 204 800_,Bracket K_R 204 801 or more_,Suburbs,Percentage of Salaries Greater than R12 800,Suburb Status,latitude,longitude
1,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Blue Hills,46.295662,Qualifies,-25.938216,28.104366
2,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Country View,46.295662,Qualifies,-25.945718,28.120936
3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Erand Gardens,46.295662,Qualifies,-25.973828,28.124027
5,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Sagewood,46.295662,Qualifies,-26.117894,28.154328
6,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,9.051092,5.414020,4.239121,Summerset,46.295662,Qualifies,-25.960780,28.108545
11,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Broadacres,35.716891,Qualifies,-26.000187,27.982335
12,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Chartwell,35.716891,Qualifies,-26.167513,28.052286
13,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Dainfern,35.716891,Qualifies,-25.993149,28.004047
15,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Kengies,35.716891,Qualifies,-25.998480,27.997580
17,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,8.784493,5.663297,4.958150,Riverbend,35.716891,Qualifies,-26.029844,27.962085


### Foursquare
Now that we have our locations for our candidate suburbs, let's use Foursquare API to get info on restaurants in each suburbs.

We will get venues for each of the suburbs

We will also get schools in and around each of the locations in question

In [73]:
#initiate foursquare credentials
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
ACCESS_TOKEN = '...'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K23IPADKIQXOTQMHWIAW1MSEU2ZF2QDC52BX23F1WMF4DNEV
CLIENT_SECRET:NL2WYW14VVDY4ZYW5MUUOM4WGPPIXGGZ0KEQULS5C4KVLFP1t


In [75]:
#define function to explore venues in each of the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,ACCESS_TOKEN,VERSION,lat,lng,radius,LIMIT)
       
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [76]:
#call function to obtain venues
suburb_venues = getNearbyVenues(names=qualifying_suburbs['Suburbs'],
                                   latitudes=qualifying_suburbs['latitude'],
                                   longitudes=qualifying_suburbs['longitude']
                                  )
suburb_venues

Blue Hills
Country View
Erand Gardens
Sagewood
Summerset
Broadacres
Chartwell
Dainfern
Kengies
Riverbend
Beverley
Fourways
Kleve
Witkoppen
Paulshof
Sunninghill
Randjesfontein
Randjespark


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Sagewood,-26.117894,28.154328,Gloria Jean's Coffees,-26.117898,28.153783,Coffee Shop
1,Broadacres,-26.000187,27.982335,Spar,-26.001034,27.982533,Supermarket
2,Broadacres,-26.000187,27.982335,Woolworths,-25.999479,27.982516,Supermarket
3,Broadacres,-26.000187,27.982335,D'Ouwe Werf,-26.000773,27.982004,Breakfast Spot
4,Broadacres,-26.000187,27.982335,GardenShop,-26.000053,27.982434,Garden Center
5,Broadacres,-26.000187,27.982335,Mugg & Bean,-25.999519,27.982542,Coffee Shop
6,Broadacres,-26.000187,27.982335,Readers Warehouse,-26.000483,27.983088,Bookstore
7,Broadacres,-26.000187,27.982335,Steers,-26.000790,27.983013,Burger Joint
8,Broadacres,-26.000187,27.982335,Sorbet,-25.999903,27.982963,Cosmetics Shop
9,Broadacres,-26.000187,27.982335,Conversations Restaurant,-26.000633,27.982438,Coffee Shop


In [77]:
#encode the venues in each suburb to be columns to allow counting of each type of venue for each suburb
suburb_venues_onehot=[]

# one hot encoding
suburb_venues_onehot = pd.get_dummies(suburb_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column and move it to the beginning of the dataframe

suburb_venues_onehot["Suburb"]=suburb_venues['Suburb'] 
first_col= suburb_venues_onehot.pop('Suburb') 
suburb_venues_onehot.insert(0,'Suburb',first_col)

suburb_venues_onehot.head()

,Suburb,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,Sagewood,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,Broadacres,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,Broadacres,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Broadacres,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Broadacres,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [78]:
suburb_venues_grouped = suburb_venues_onehot.groupby('Suburb').mean().reset_index()
suburb_venues_grouped

,Suburb,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,Broadacres,0.1,0.1,0.1,0.2,0.0,0.1,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.2,0.0
1,Chartwell,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.2,0.0,0.0
2,Sagewood,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sunninghill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Witkoppen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Define function to obtain schools in each of the suburbs

In [90]:
def getNearbySchools(names, latitudes, longitudes, radius=500,query="school"):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(CLIENT_ID,CLIENT_SECRET,ACCESS_TOKEN,VERSION,lat,lng,radius,LIMIT,query)
       
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(results)
        
        # return only relevant information for each nearby schools
        venues_list.append([(name, lat, lng, v['venue']['name'],v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue','Category' ]
    
    return(nearby_venues)

In [91]:
#call function to obtain schools
suburb_schools = getNearbySchools(names=qualifying_suburbs['Suburbs'],
                                   latitudes=qualifying_suburbs['latitude'],
                                   longitudes=qualifying_suburbs['longitude']
                                  )
suburb_schools

Blue Hills
[]
Country View
[]
Erand Gardens
[]
Sagewood
[]
Summerset
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d2b40f4d86aa090229f22c0', 'name': 'Sagewood school', 'location': {'address': 'sagewood avenue', 'lat': -25.957090324684323, 'lng': 28.10866452438543, 'labeledLatLngs': [{'label': 'display', 'lat': -25.957090324684323, 'lng': 28.10866452438543}], 'distance': 410, 'postalCode': '1687', 'cc': 'ZA', 'city': 'Midrand', 'state': 'IGauteng', 'country': 'iNingizimu Afrika', 'formattedAddress': ['sagewood avenue', 'midrand', '1687']}, 'categories': [{'id': '4bf58dd8d48988d13b941735', 'name': 'School', 'pluralName': 'Schools', 'shortName': 'School', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4d2b40f4d86aa090229f22c0-0'}, {'reasons': {'count': 0, 'it

,Suburb,Suburb Latitude,Suburb Longitude,Venue,Category
0,Summerset,-25.960780,28.108545,Sagewood school,School
1,Summerset,-25.960780,28.108545,Summerhill Prep,School
2,Chartwell,-26.167513,28.052286,Bokamoso day care center,Nursery School
3,Riverbend,-26.029844,27.962085,Impact Kidz,School
4,Witkoppen,-26.007500,28.008611,Junior college,Nursery School
5,Paulshof,-26.030556,28.049167,Nova Pioneer - Paulshof,Elementary School
6,Randjesfontein,-25.948278,28.145213,Hedgehog Meadow Nursery School,School


In [92]:
#encode the venues in each suburb to be columns to allow counting of each type of venue for each suburb

suburb_schools_onehot=[]

# one hot encoding
suburb_schools_onehot = pd.get_dummies(suburb_schools[['Category']], prefix="", prefix_sep="")

# add neighborhood column and move it to the beginning of the dataframe

suburb_schools_onehot["Suburb"]=suburb_schools['Suburb'] 
first_col= suburb_schools_onehot.pop('Suburb') 
suburb_schools_onehot.insert(0,'Suburb',first_col)

suburb_schools_onehot.head()

,Suburb,Elementary School,Nursery School,School
0,Summerset,0,0,1
1,Summerset,0,0,1
2,Chartwell,0,1,0
3,Riverbend,0,0,1
4,Witkoppen,0,1,0


In [93]:
suburb_schools_grouped = suburb_schools_onehot.groupby('Suburb').mean().reset_index()
suburb_schools_grouped

,Suburb,Elementary School,Nursery School,School
0,Chartwell,0,1,0
1,Paulshof,1,0,0
2,Randjesfontein,0,0,1
3,Riverbend,0,0,1
4,Summerset,0,0,1
5,Witkoppen,0,1,0


In [94]:
suburb_schools_grouped.drop(["Elementary School","Nursery School"],axis=1,inplace=True)
suburb_schools_grouped

,Suburb,School
0,Chartwell,0
1,Paulshof,0
2,Randjesfontein,1
3,Riverbend,1
4,Summerset,1
5,Witkoppen,0


In [95]:
#merge the dataframes with data on schools and venues for each suburb
schools_and_restaurants_df =suburb_schools_grouped.join(suburb_venues_grouped.set_index("Suburb"),on="Suburb")
schools_and_restaurants_df

,Suburb,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,Chartwell,0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.2,0.0,0.0
1,Paulshof,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Randjesfontein,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Riverbend,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Summerset,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Witkoppen,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [96]:
schools_and_restaurants_df=schools_and_restaurants_df.fillna(0)
schools_and_restaurants_df

,Suburb,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop,Construction & Landscaping,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,Chartwell,0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.2,0.0,0.0
1,Paulshof,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Randjesfontein,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Riverbend,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Summerset,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Witkoppen,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


We now merge the dataframe with schools and venues with the one with economic data for each suburb

In [109]:
qualifying_suburbs = qualifying_suburbs.rename(columns={"Suburbs":"Suburb"})

In [110]:
schools_and_restaurants_economy_df = qualifying_suburbs.join(schools_and_restaurants_df.set_index("Suburb"),on="Suburb")
schools_and_restaurants_economy_df.reset_index(drop=True,inplace=True)

In [111]:
schools_and_restaurants_economy_df.dropna(axis=0,inplace=True)
schools_and_restaurants_economy_df.reset_index(drop=True,inplace=True)
schools_and_restaurants_economy_df

,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,Bracket H_R 25 601 to R 51 200_,...,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,13.009046,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,...,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.2,0.0,0.0
2,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,8.756917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,94.0,10.405336,2.608327,2.110618,3.298897,5.208601,4.138641,6.247244,8.478387,8.046029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,93.0,10.288936,2.139376,2.298091,3.517142,4.189170,4.750043,6.680643,8.912124,10.652979,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,92.0,19.644520,11.833525,5.300199,7.016788,9.400848,8.690430,7.768262,7.805791,7.698323,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Methodology <a name="methodology"></a>

A clustering algorithm will be used to group the list of suburbs that have been identified based on the features of economic data, number of schools in the area the area as well as types and number of restaurants in the area


In [112]:
#import required libraries
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [113]:
#scale the data to allow proper clustering without being affected by weights of the economic data
# define standard scaler
scaler = StandardScaler()
# transform data
scaled_sre_df = scaler.fit_transform(schools_and_restaurants_economy_df.drop(['Percentage of Salaries Greater than R12 800','Suburb', 'Suburb Status','latitude','longitude'],axis=1))

suburb_grouped_clustering = scaled_sre_df 
suburb_grouped_clustering


array([[ 2.18376143,  0.89310247, -0.31750242, -0.05061261,  0.45186609,
        -0.19838568,  0.22919634,  1.86114245,  2.19085925,  1.92416932,
         0.92859517,  0.68339449,  0.23525096,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.4472136 ,  0.        ,
         0.        , -0.4472136 ,  0.        , -0.4472136 ,  0.        ,
        -0.4472136 , -0.4472136 ,  0.        , -0.4472136 ],
       [-0.17175652, -0.95045312, -0.42880442, -0.35534123, -0.60115606,
        -0.65558925, -0.84450172, -1.0451312 , -0.64249537, -0.39866342,
         0.72057209,  0.92910028,  1.05085075, -1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  2.23606798,  0.        ,
         0.        ,  2.23606798,  0.        ,  2.23606798,  0.        ,
         2.23606798,  2.23606798,  0.        , -0.4472136 ],
       [-0.17175652, -0.95045312, -0.42880442, -0.35534123, -0.60115606,
        -0.65558925, -0.84450172, -1.0451312 , -0.64249537, -0.39866342,
  

In [114]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(init="k-means++",n_clusters=kclusters, random_state=0,n_init=5).fit(suburb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

array([3, 2, 0, 0, 0])

In [115]:
schools_and_restaurants_economy_df.insert(0, 'Cluster Labels', kmeans.labels_)
schools_and_restaurants_economy_df

,Cluster Labels,Ward,No income,Bracket A_R 1 to R 400_,Bracket B_R 401 to R 800_,Bracket C_R 801 to R 1 600_,Bracket D_R 1 601 to R 3 200_,Bracket E_R 3 201 to R 6 400_,Bracket F_R 6 401 to R 12 800_,Bracket G_R 12 801 to R 25 600_,...,Cosmetics Shop,Garden Center,Italian Restaurant,Park,Pharmacy,Pizza Place,Shoe Store,Snack Place,Supermarket,Yoga Studio
0,3,112.0,15.865469,3.092353,2.909317,4.880223,4.950943,5.399016,10.320785,14.582383,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,...,0.0,0.0,0.2,0.0,0.2,0.0,0.2,0.2,0.0,0.0
2,0,96.0,7.860810,2.710250,2.581954,3.474502,4.093808,3.477383,4.748374,7.554034,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,94.0,10.405336,2.608327,2.110618,3.298897,5.208601,4.138641,6.247244,8.478387,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,93.0,10.288936,2.139376,2.298091,3.517142,4.189170,4.750043,6.680643,8.912124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,92.0,19.644520,11.833525,5.300199,7.016788,9.400848,8.690430,7.768262,7.805791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Analysis <a name="analysis"></a>

With the clustering is done, the dataframe will be sorted by most popular venues to assist in the analysis by the clustering label to determine which label identifies the ost suitable suburb

In [116]:
#define function to sort suburbs
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [117]:
#get column names to define sorting
schools_and_restaurants_economy_df.columns

Index(['Cluster Labels', 'Ward', 'No income', 'Bracket A_R 1 to R 400_',
       'Bracket B_R 401 to R 800_', 'Bracket C_R 801 to R 1 600_',
       'Bracket D_R 1 601 to R 3 200_', 'Bracket E_R 3 201 to R 6 400_',
       'Bracket F_R 6 401 to R 12 800_', 'Bracket G_R 12 801 to R 25 600_',
       'Bracket H_R 25 601 to R 51 200_', 'Bracket I_R 51 201 to R 102 400_',
       'Bracket J_R 102 401 to R 204 800_', 'Bracket K_R 204 801 or more_',
       'Suburb', 'Percentage of Salaries Greater than R12 800',
       'Suburb Status', 'latitude', 'longitude', 'School', 'Bookstore',
       'Breakfast Spot', 'Burger Joint', 'Coffee Shop',
       'Construction & Landscaping', 'Cosmetics Shop', 'Garden Center',
       'Italian Restaurant', 'Park', 'Pharmacy', 'Pizza Place', 'Shoe Store',
       'Snack Place', 'Supermarket', 'Yoga Studio'],
      dtype='object')

In [120]:
#get index nmbers for the columns
index_no=[]

columns = ['Suburb',
       'School','Bookstore',
       'Breakfast Spot', 'Burger Joint', 'Coffee Shop',
       'Construction & Landscaping', 'Cosmetics Shop', 'Garden Center',
       'Italian Restaurant', 'Park', 'Pharmacy', 'Pizza Place', 'Shoe Store',
       'Snack Place', 'Supermarket', 'Yoga Studio']


for column in columns:
    index_no.append(schools_and_restaurants_economy_df.columns.get_loc(column ))

index_no

[14, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]

In [121]:
#sort most common venues in each suburb
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
suburb_venues_sorted = pd.DataFrame(columns=columns)
suburb_venues_sorted['Suburb'] = schools_and_restaurants_economy_df['Suburb']

for ind in np.arange(schools_and_restaurants_df.shape[0]):
    suburb_venues_sorted.iloc[ind, 1:] = return_most_common_venues(schools_and_restaurants_economy_df.iloc[ind,index_no], num_top_venues)


In [122]:
suburb_venues_sorted

,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Summerset,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
1,Chartwell,Construction & Landscaping,Italian Restaurant,Pharmacy,Shoe Store,Snack Place
2,Riverbend,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
3,Witkoppen,Yoga Studio,School,Bookstore,Breakfast Spot,Burger Joint
4,Paulshof,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
5,Randjesfontein,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop


In [123]:
#insert the cluster labels as well as the coordinates to allow analysis 
suburb_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
suburb_venues_sorted.insert(0,'latitude',schools_and_restaurants_economy_df['latitude'])
suburb_venues_sorted.insert(0,'longitude',schools_and_restaurants_economy_df['longitude'])
suburb_venues_sorted

,longitude,latitude,Cluster Labels,Suburb,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,28.108545,-25.960780,3,Summerset,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
1,28.052286,-26.167513,2,Chartwell,Construction & Landscaping,Italian Restaurant,Pharmacy,Shoe Store,Snack Place
2,27.962085,-26.029844,0,Riverbend,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
3,28.008611,-26.007500,0,Witkoppen,Yoga Studio,School,Bookstore,Breakfast Spot,Burger Joint
4,28.049167,-26.030556,0,Paulshof,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop
5,28.145213,-25.948278,1,Randjesfontein,School,Bookstore,Breakfast Spot,Burger Joint,Coffee Shop


## Results and Discussion<a name="results"></a>

The only area without a burger joint is chartwell, but it doesnt have a lot of restaurants in the area, so may not be ideal exactly. The second most ideal place is Witkoppen, where burger joints are the least common place. In general, it looks like suburbs with cluster label 0 are the suitable ones

We mow map the areas

In [127]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 

latitude = -26.1
longitude =28

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(suburb_venues_sorted['latitude'], suburb_venues_sorted['longitude'], suburb_venues_sorted['Suburb'],suburb_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusion <a name="conclusion"></a>

The purpose of the project was to identify the most suitable locations for a burger restaurant. This was done using a clustering algorithm which identified the label which shows the most suitable suburbs for a burger restaurant.

In the end, a number of suburbs we classified with this label, but only one was identified as the most suitable.